# Results

In [1]:
!nvidia-smi

Tue Apr  4 12:10:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   59C    P0   263W / 300W |  19905MiB / 32768MiB |     88%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:62:00.0 Off |                    0 |
| N/A   

## Load Dependencies

In [1]:
from collections import defaultdict
import IPython.display as ipd
import os, sys
from pathlib import Path
import torch
import yaml
import soundfile as sf
from parallel_wavegan.utils import load_model
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '1'  # Specify your cuda device here
PROJECT_ROOT = "/project/fdreyer/projects/vqvae-vc"
sys.path.append(PROJECT_ROOT)
%cd {PROJECT_ROOT}

from src.models.hle_vqvae_vc import HleVqVaeVc
from src.data.datamodules import VCDataModule
from src.params import global_params

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Use {device} device")

sr = 24_000

/project/fdreyer/projects/vqvae-vc
Use cuda device


Vocoder:

In [2]:
vocoder = load_model(global_params.PATH_HIFIGAN_PARAMS).to(device)

Validation Dataset:

In [3]:
VCTK_DATASET_CONFIG_PATH = os.path.join(PROJECT_ROOT, "config", "data", "esd_eng", "esd_eng-24kHZ.yml")
with open(VCTK_DATASET_CONFIG_PATH) as f:
    vctk_config = yaml.safe_load(f)["data"]

vctk_data_module = VCDataModule(**vctk_config)
vctk_data_module.prepare_data()
vctk_val_data_loader = vctk_data_module.val_dataloader()
vctk_train_data_loader = vctk_data_module.train_dataloader()

Create Train Dataset
Create Val Dataset


## HLE-VQVAE-VC

In [8]:
HLE_VQVAE_CHECKPOINT_DIR = os.path.join(PROJECT_ROOT, "lightning_logs", "version_124", "checkpoints")
checkpoint_path = os.path.join(HLE_VQVAE_CHECKPOINT_DIR, os.listdir(HLE_VQVAE_CHECKPOINT_DIR)[0])
vqvae_vc = HleVqVaeVc.load_from_checkpoint(checkpoint_path)
vqvae_vc.to(device)

HleVqVaeVc(
  (speaker_embedding): SpeakerEmbedding(
    (embedding): Embedding(10, 16)
  )
  (encoder_bot): HleEncoder(
    (input_layer): Sequential(
      (0): HleConv1d(80, 256, kernel_size=(15,), stride=(1,), padding=(7,))
      (1): GLU(dim=1)
    )
    (downsampling_layer): Sequential(
      (0): HleConv1d(128, 256, kernel_size=(8,), stride=(2,), padding=(3,))
      (1): InstanceNorm1d(256, eps=1e-05, momentum=0.8, affine=False, track_running_stats=False)
      (2): GLU(dim=1)
    )
    (dilation_stack): WaveNetLikeStack(
      (stack): ModuleList(
        (0): WaveNetLikeCell(
          (in_seq_layer): Sequential(
            (0): HleConv1d(128, 256, kernel_size=(5,), stride=(1,), padding=(2,))
            (1): InstanceNorm1d(256, eps=1e-05, momentum=0.25, affine=False, track_running_stats=False)
          )
          (residual_layer): Sequential(
            (0): HleConv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
            (1): InstanceNorm1d(128, eps=1e-05, mo

Voice Conversion:

In [ ]:
n_speakers = 10

speaker_utterances = defaultdict(lambda: [])
with torch.no_grad():
    for batch in tqdm(vctk_val_data_loader):
        original_mels, original_wavs, original_speakers, _ = batch
        original_mels = original_mels.to(device)
        all_converted_mels = [[] for _ in range(original_mels.size(0))]
        all_converted_wavs = [[] for _ in range(original_wavs.size(0))]
        for converted_speaker in range(n_speakers):
            converted_speakers = torch.ones((original_mels.size(0),), dtype=torch.long).to(device) * converted_speaker
            converted_mels, _, _ = vqvae_vc(original_mels, converted_speakers)
            for idx, converted_mel in enumerate(converted_mels):
                all_converted_mels[idx].append(converted_mel.cpu().numpy())
                all_converted_wavs[idx].append(vocoder.inference(converted_mel.transpose(-1, -2).squeeze()).squeeze().detach().cpu().numpy())
        for original_mel, original_wav, original_speaker, converted_mels, converted_wavs in zip(original_mels, original_wavs, original_speakers, all_converted_mels, all_converted_wavs):
            speaker_utterances[original_speaker.item()].append({
                "original_mel": original_mel.cpu().numpy(),
                "original_wav": original_wav.numpy(),
                "converted_mels": converted_mels,
                "converted_wavs": converted_wavs
            })

 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                             | 18/24 [11:09<03:43, 37.18s/it]

Listen to original and converted audios:

In [ ]:
speaker = 0
utterance = 10

utterance_dict = speaker_utterances[speaker][utterance]
print("Original: ")
display(ipd.Audio(utterance_dict["original_wav"], rate=sr))
print("Conversions: ")
for converted_wav in utterance_dict["converted_wavs"]:
    display(ipd.Audio(converted_wav, rate=sr))

Save originals and voice converted audios:

In [149]:
model = "hle-vqvae-vc"

out_dir = os.path.join(".", "reports", "audios", model)
for speaker, utterances in tqdm(speaker_utterances.items()):
    out_dir_speaker = os.path.join(out_dir, f"speaker{str(speaker)}")
    for utterance_idx, utterance in enumerate(utterances):
        out_dir_speaker_utterance = os.path.join(out_dir_speaker, f"utterance{str(utterance_idx)}")
        Path(out_dir_speaker_utterance).mkdir(parents=True, exist_ok=True)
        original_wav = utterance["original_wav"]
        sf.write(os.path.join(out_dir_speaker_utterance, f"utterance{utterance_idx}.speaker{speaker}.original.wav"), original_wav, sr, format="wav")
        converted_wavs = utterance["converted_wavs"]
        for target_speaker, converted_wav in enumerate(converted_wavs):
            sf.write(os.path.join(out_dir_speaker_utterance, f"utterance{utterance_idx}.speaker{speaker}-to-speaker{target_speaker}.wav"), converted_wav, sr, format="wav")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:06<00:00,  6.67s/it]
